In [1]:
import pandas as pd
from copy import deepcopy
import datetime
import numpy as np
import data_preprocessing.features_generations.ts_lag_features_generator as lag_gen

In [2]:
date_col = 'DatetimeEnd'
filter_col = 'NoFilter'
target_cols = ['AQI', 'AQI_O3', 'AQI_PM25']
source_file = "datasets_tests/pollutants-merged-data/pol_merged.csv"
data = pd.read_csv(source_file, index_col=date_col, parse_dates=True)
data[filter_col] = 1
data = data.sort_values(date_col)
group_col = [filter_col] + [] + [date_col]
preagg_method = 'mean'
lag = 7
data.head()

,AQI_O3,AQI_PM25,AQI,Pollutant,C_MEAN_O3,C_MEAN_PM25,NoFilter
DatetimeEnd,,,,,,,
2023-01-01,27,68,68,6001,51.766087,20.416739,1
2023-01-02,23,39,39,6001,38.189583,9.294000,1
2023-01-03,25,60,60,6001,32.061667,16.974417,1
2023-01-04,31,51,51,6001,59.843636,12.758045,1
2023-01-05,27,74,74,6001,43.312500,23.074208,1


In [3]:
data_cl = deepcopy(data)
data_cl.reset_index(inplace=True)
data_cl[date_col] = data_cl[date_col].map(datetime.datetime.date)
data_cl.set_index(date_col, inplace=True)
data_cl.sort_values(date_col, inplace=True)

In [4]:
data_preag_filled = lag_gen.calc_preag_fill(data_cl, group_col, date_col, target_cols, preagg_method)
data_preag_filled.head()

,NoFilter,DatetimeEnd,AQI,AQI_O3,AQI_PM25
0,1,2023-01-01,68.0,27.0,68.0
1,1,2023-01-02,39.0,23.0,39.0
2,1,2023-01-03,60.0,25.0,60.0
3,1,2023-01-04,51.0,31.0,51.0
4,1,2023-01-05,74.0,27.0,74.0


In [5]:
alpha = 7
ewm_filled = lag_gen.calc_ewm(data_preag_filled, group_col, date_col, alpha)
ewm = lag_gen.shift(ewm_filled, group_col, date_col, lag)
ewm

,NoFilter,DatetimeEnd,AQI,AQI_O3,AQI_PM25
0,1,2023-01-01,NaN,NaN,NaN
1,1,2023-01-02,NaN,NaN,NaN
2,1,2023-01-03,NaN,NaN,NaN
3,1,2023-01-04,NaN,NaN,NaN
4,1,2023-01-05,NaN,NaN,NaN
...,...,...,...,...,...
92,1,2023-04-03,36.001200,34.912400,21.938009
93,1,2023-04-04,35.500900,34.684300,23.703507
94,1,2023-04-05,35.375675,34.763225,24.527630
95,1,2023-04-06,36.531756,36.072419,25.645723


In [13]:
s = 'percentileL(10)'
l = len('percentile')
n = s[:l]
import re
re.findall(r'\d+', s)[0]

'10'

In [15]:
lf_df = ewm_filled.apply(lambda x: x.shift(lag)).reset_index()
#data_cl[date_col].map(datetime.datetime.date)
#lf_df[date_col] = pd.to_datetime(lf_df[date_col].astype(str))
lf_df[date_col] = pd.to_datetime(lf_df[date_col].astype(str)).map(datetime.datetime.date)


TypeError: 'Series' object is not callable

In [13]:
lf_df = ewm_filled.apply(lambda x: x.shift(lag)).reset_index()

In [ ]:
ewm_filled = calc_ewm(data_preag_filled, group_col, date_col, alpha)
ewm = shift(ewm_filled, group_col, date_col, lag)
new_names = \
    {x: f"{x}_lag{lag}d_ewm{alpha}_{key_str}{preagg_str}{filter_col_str}"
     for x in target_cols}
data_cl = pd.merge(data_cl, ewm.rename(columns=new_names),
                   how='left', on=group_col)

In [ ]:
lf_df[date_col] = pd.to_datetime(lf_df[date_col].astype(str))
ewm = shift(ewm_filled, group_col, date_col, lag)

In [35]:
min_date, max_date = data_preag[date_col].min(), data_preag[date_col].max()
groupby_day = data_preag.groupby(pd.PeriodIndex(data_preag[date_col], freq='D'))
results = groupby_day.sum(min_count=1)
idx = pd.period_range(min_date, max_date)
results = results.reindex(idx, fill_value=np.nan)
if date_col in results.columns.values:
    results.drop([date_col], axis=1, inplace=True)



,NoFilter,AQI,AQI_O3,AQI_PM25
2023-01-01,1,68.0,27.0,68.0
2023-01-02,1,39.0,23.0,39.0
2023-01-03,1,60.0,25.0,60.0
2023-01-04,1,51.0,31.0,51.0
2023-01-05,1,74.0,27.0,74.0
...,...,...,...,...
2023-04-03,1,36.0,36.0,14.0
2023-04-04,1,39.0,39.0,33.0
2023-04-05,1,63.0,39.0,63.0
2023-04-06,1,51.0,28.0,51.0


In [17]:
min_date, max_date = data_preag[date_col].min(), data_preag[date_col].max()
groupby_day = data_preag.groupby(pd.PeriodIndex(data_preag[date_col], freq='D'))
results = groupby_day.sum(min_count=1)

idx = pd.period_range(min_date, max_date)
results = results.reindex(idx, fill_value=np.nan)

results.index.rename(date_col, inplace=True)

KeyError: 'DatetimeEnd'

In [15]:
data_preag_filled = data_preag.groupby(group_col[:-1]).apply(
    fill_missing_dates, date_col=date_col).drop(group_col[:-1],
                                                axis=1).reset_index()

ValueError: cannot insert DatetimeEnd, already exists

In [41]:
data_preag = out_df.groupby(group_col).agg(preagg)[target_cols].reset_index()
min_date, max_date = data_preag[date_col].min(), data_preag[date_col].max()
groupby_day = data_preag.groupby(pd.PeriodIndex(data_preag[date_col], freq='D'))

In [43]:
results = groupby_day.sum(min_count=1)

In [27]:
dc = data.reset_index()
import datetime
dc[date_col] = dc[date_col].map(datetime.datetime.date)

In [33]:
results = groupby_day.sum(min_count=1)

In [ ]:
idx = pd.period_range(min_date, max_date)
results = results.reindex(idx, fill_value=np.nan)
results.index.rename(date_col, inplace=True)

In [37]:
data_preag_filled = lag_gen.calc_preag_fill(out_df, group_col, date_col, target_cols, preagg)

ValueError: cannot insert DatetimeEnd, already exists